In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [6]:
# Load Dataset
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding = "latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
df = df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

print(df.shape)

(5572, 2)


In [9]:
# encoding catogerical variable column v1
df['v1'] = df["v1"].map({'spam':1,'ham':0})
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

There are **no missing values** in records

In [10]:
df["v1"].value_counts()

0    4825
1     747
Name: v1, dtype: int64

This set needs to be balanced.

In [13]:
from sklearn.utils import resample

# create two different dataframe of majority and minority class 

df_majority = df[(df['v1'] == 0)] 

df_minority = df[(df['v1'] == 1)] 

# upsample minority class
df_minority_upsampled = resample(df_minority, replace = True, n_samples = df_majority.shape[0], random_state = 42) 

# reproducible results
    
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# Convert text into vectors using bag of words
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df_upsampled["v2"])

y = df_upsampled["v1"]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.3, random_state=0)

In [16]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

gaussian_clf = GaussianNB()

gaussian_clf.fit(X_train, y_train)

gaussian_clf.score(X_test, y_test)

0.9395509499136442

In [17]:
multinomial_clf = MultinomialNB()

multinomial_clf.fit(X_train, y_train)

multinomial_clf.score(X_test, y_test)

0.9858376511226252

In [21]:
from sklearn.metrics import confusion_matrix
m_confusion_test = confusion_matrix(y_test, gaussian_clf.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted ham', 'Predicted spam'],
            index = ['Actual ham', 'Actual spam'])

,Predicted ham,Predicted spam
Actual ham,1259,175
Actual spam,0,1461


In [22]:
m_confusion_test = confusion_matrix(y_test, multinomial_clf.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted ham', 'Predicted spam'],
            index = ['Actual ham', 'Actual spam'])

,Predicted ham,Predicted spam
Actual ham,1411,23
Actual spam,18,1443
